In [1]:
from os.path import join
from glob import glob
import numpy as np
import dask
import dask.array as da
import pyklb as klb
import napari

# 2701 timepoints
#Shape of each 3D stack is (75,768,1848)

D:\Anaconda3\envs\denoising\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
D:\Anaconda3\envs\denoising\lib\site-packages\napari\__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
#Use opportunistic caching 
from dask.cache import Cache
cache = Cache(16e9)  # Set aside some memory
cache.register()    # Turn cache on globally

In [3]:
inPath = 'W:/SV3/RC_15-06-11/Dme_E2_His2AvRFP_spiderGFP_12-03_20150611_155054.corrected/Results/TimeFused/'
fnames = glob(join(inPath, '*/*CM01_CHN01.fusedStack.klb' ))
fnames.sort()
fnames = fnames[0:100]
nviews = 1

In [4]:
sample = klb.readfull(fnames[0]) #Sample image

# Generate lazy arrays
lazy_values = [dask.delayed(klb.readfull)(fn) for fn in fnames] #Lazily evaluate klb.readfull on each klb file
lazy_arrays = [da.from_delayed(lazy_value,                      #Construct a dask array for every lazy value
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values]

In [5]:
#Generate empty object array to organize each chunk that loads the 3D volume
#stack = np.empty((nviews,len(fnames)/nviews,1,1,1), dtype=object) #Dimension of (view,timepoint,Z,Y,X)
stack = np.empty((len(fnames),1,1,1), dtype=object) #Dimension of (view,timepoint,Z,Y,X)

for fn, x in zip(fnames, lazy_arrays):
    #view = int(fn[fn.index("_CM")+3:].split("_")[0])
    timepoint = int(fn[fn.index("_TM")+3:].split("_")[0])
    #stack[view,timepoint,0,0,0] = x
    stack[timepoint,0,0,0] = x
    
#Stitch together all these blocks into a single N-dimensional array
stacks = da.block(stack.tolist())

#Visual of the dask array
stacks

,Array,Chunk
Bytes,21.29 GB,212.89 MB
Shape,"(100, 75, 768, 1848)","(1, 75, 768, 1848)"
Count,400 Tasks,100 Chunks
Type,uint16,numpy.ndarray


In [6]:
#Rechunk the stacks

#stacks = stacks.rechunk((nviews,1,8,768,1848))
stacks = stacks.rechunk((1,8,768,1848))

#Visual of the dask array
stacks

,Array,Chunk
Bytes,21.29 GB,22.71 MB
Shape,"(100, 75, 768, 1848)","(1, 8, 768, 1848)"
Count,2500 Tasks,1000 Chunks
Type,uint16,numpy.ndarray


In [8]:
%gui qt5

In [11]:
# create a Viewer and add the image
#viewer = napari.Viewer(axis_labels = ['view','t','z','y','x'])
viewer = napari.Viewer(axis_labels = ['t','z','y','x'])
viewer.add_image(stacks,
                 scale = [1,1,8,1,1],
                 colormap='inferno',
                 blending='additive',
                 name='nuclei',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,180])

<Image layer 'nuclei' at 0x2d7b09e9b88>

### Observations

Note: 16 GB of cache was allocated for these tests.

The interactivity of the napari browser is good in 'z' and has some time lags in 't', under these scenarios:
- 100 timepoints (21 GB), chunking 8 z-planes (23 MB chunks)